In [ ]:
! pip install duckdb
! pip install pandas
! pip install seaborn
! pip install matplotlib
! pip install jupysql pandas matplotlib duckdb-engine

In [7]:
import duckdb 
import pandas as pd
%load_ext sql
conn = duckdb.connect()
%sql conn --alias duckdb
%sql INSTALL spatial; LOAD spatial;
%config SqlMagic.displaylimit = None;

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


Running query in 'duckdb'

displaylimit: Value None will be treated as 0 (no limit)

In [16]:
df = pd.read_excel('../Protocols.xlsx', sheet_name=None )
df = pd.concat(df.values())


In [22]:
%%sql
COPY (
SELECT distinct organization_name, organization_id
FROM df
WHERE organization_name IS NOT NULL
)
TO 'organization.csv'

Running query in 'duckdb'

Count
5


In [5]:
%%sql
CREATE TABLE IF NOT EXISTS Oxford_protocol_material_data AS
SELECT * FROM st_read('../Materials in Protocols.xlsx', layer="Oxford Churchill");

CREATE TABLE IF NOT EXISTS Oxford_protocol_data AS
SELECT * FROM st_read('../Protocols.xlsx', layer="Oxford Churchill");



Running query in 'duckdb'

BinderException: Binder Error: Layer '*' could not be found in dataset

In [3]:
%%sql
SELECT DISTINCT Field31 FROM Oxford_protocol_material_data

Running query in 'duckdb'

Field31
None
Mr.
Stephen


In [4]:
%%sql
WITH Oxford_data_column_surgeon AS (
    SELECT procedure_id, protocol_id, IFNULL( surgeon_name || '|' || surgeon_surname, 'Base') AS Surgeon, material_id, material_name, surgeon_name
    FROM Oxford_protocol_material_data OGP
    ORDER BY procedure_id, protocol_id, surgeon_name NULLS FIRST, surgeon_surname 
),
Oxford_data_default AS (
    SELECT * 
    FROM Oxford_data_column_surgeon
    WHERE surgeon_name IS NULL
),
Oxford_data_surgeon AS (
    SELECT * 
    FROM Oxford_data_column_surgeon
    WHERE surgeon_name IS NOT NULL
)
SELECT *
FROM Oxford_data_default ODSU
LEFT OUTER JOIN Oxford_data_surgeon ODST ON 
    ODSU.procedure_id = ODST.procedure_id AND
    ODSU.protocol_id = ODST.protocol_id AND
    ODSU.material_id = ODST.material_id
WHERE ODSU.protocol_id = 9219.0
ORDER BY ODSU.Surgeon




Running query in 'duckdb'

procedure_id,protocol_id,Surgeon,material_id,material_name,surgeon_name,procedure_id_1,protocol_id_1,Surgeon_1,material_id_1,material_name_1,surgeon_name_1
1493.0,9219.0,Base,81976.0,Jug,None,1493.0,9219.0,William|Gietzmann,81976.0,Jug,William
1493.0,9219.0,Base,83499.0,5mm Endo Clip,None,1493.0,9219.0,William|Gietzmann,83499.0,5mm Endo Clip,William
1493.0,9219.0,Base,82011.0,Gallipot,None,1493.0,9219.0,William|Gietzmann,82011.0,Gallipot,William
1493.0,9219.0,Base,82056.0,Back support,None,1493.0,9219.0,William|Gietzmann,82056.0,Back support,William
1493.0,9219.0,Base,82092.0,Pillow,None,1493.0,9219.0,William|Gietzmann,82092.0,Pillow,William
1493.0,9219.0,Base,82162.0,Urometer,None,1493.0,9219.0,William|Gietzmann,82162.0,Urometer,William
1493.0,9219.0,Base,82242.0,Hemolok-GOLD 10mm,None,1493.0,9219.0,William|Gietzmann,82242.0,Hemolok-GOLD 10mm,William
1493.0,9219.0,Base,82243.0,Hemolok-Green 5mm,None,1493.0,9219.0,William|Gietzmann,82243.0,Hemolok-Green 5mm,William
1493.0,9219.0,Base,82267.0,Marker pen,None,1493.0,9219.0,William|Gietzmann,82267.0,Marker pen,William
1493.0,9219.0,Base,82640.0,Light Handles,None,1493.0,9219.0,William|Gietzmann,82640.0,Light Handles,William


In [ ]:
%%sql    
SELECT procedure_id, protocol_id, surgeon_name || '|' || surgeon_surname AS Surgeon, material_id, material_name
    FROM Oxford_protocol_material_data OGP
    WHERE protocol_id = 7160.0
    ORDER BY procedure_id, protocol_id, surgeon_name NULLS FIRST, surgeon_surname 

In [ ]:
%%sql   
SELECT procedure_id, protocol_id, COALESCE(surgeon_name, concat(surgeon_name,surgeon_surname), NULL) AS Surgeon, surgeon_surname, surgeon_name, material_id, material_name
FROM Oxford_protocol_material_data OGP
WHERE protocol_id = 7160.0
ORDER BY procedure_id, protocol_id, surgeon_name NULLS FIRST, surgeon_surname 